In [ ]:
import json

with open('filtered_data.json') as f:
	data = json.load(f)

print(len(data))
print(data[0])

In [ ]:
from itertools import product

amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
word_size = 3

# Generate all sequences of word_size amino acids
vocab = [''.join(seq) for seq in product(amino_acids, repeat=word_size)]

print(len(vocab))

In [ ]:
class HashMap:
	def __init__(self, elements):
		self.data = {}
		for i, e in enumerate(elements):
			self.data[e] = i + 1
	
	def __getitem__(self, key):
		return self.data[key]

	def __len__(self):
		return len(self.data)

vocab_map = HashMap(vocab)
print(vocab_map['AAA'])

In [4]:
import torch

context_length = 900 - word_size + 1

X = torch.zeros(len(data), context_length, dtype=torch.long)
L = torch.empty(len(data), dtype=torch.long)

# Convert sequences to tensors
for i, d in enumerate(data):
	sequence = d['sequence']
	L[i] = len(sequence) - word_size + 1
	for j in range(L[i]):
		word = sequence[j:j+word_size]
		X[i, j] = vocab_map[word]

In [ ]:
print(X[0])

In [6]:
# Save input data
torch.save(X, 'X.pt')
torch.save(L, 'L.pt')

In [ ]:
classes = set()

for d in data:
	for c in d['go_ids']:
		classes.add(c)

print(len(classes))

In [8]:
classes = list(classes)

# Save reverse mapping
with open('classes.json', 'w') as file:
	json.dump(classes, file)

In [ ]:
class HashMap:
	def __init__(self, elements):
		self.data = {}
		for i, e in enumerate(elements):
			self.data[e] = i
	
	def __getitem__(self, key):
		return self.data[key]

	def __len__(self):
		return len(self.data)

classes_map = HashMap(classes)
print(classes_map['GO:0003700'])

In [ ]:
Y = torch.zeros(len(data), len(classes), dtype=torch.bool)

for i, d in enumerate(data):
	for c in d['go_ids']:
		Y[i, classes_map[c]] = True

print(Y[0])

In [11]:
# Save output data
torch.save(Y, 'Y.pt')

In [ ]:
print(X.shape, Y.shape)